In [1]:
import os

In [2]:
%pwd

'a:\\mlflow\\Heart-Failure-Prediction-with-Machine-Learning-MLflow\\research'

In [3]:
os.listdir(os.getcwd())

['1.data_ingestion.ipynb',
 '2.data_validation.ipynb',
 '3.data_transformation.ipynb',
 '4.model_trainer.ipynb',
 '5.model_evaluation.ipynb',
 'logs',
 'trials.ipynb']

In [4]:
os.chdir("../")

In [5]:
%pwd

'a:\\mlflow\\Heart-Failure-Prediction-with-Machine-Learning-MLflow'

In [6]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/Rutvik46/Heart-Failure-Prediction-with-Machine-Learning-MLflow.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="Rutvik46" 
os.environ["MLFLOW_TRACKING_PASSWORD"]="fdf9d63cdff88d1ed6fd22bc2f3d34a8c0672d1a"

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir:str
    test_data_path:Path
    model_paths:dict
    all_params:dict
    metrics_file_paths: dict
    target_column:str
    mlflow_url:str

In [8]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [12]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH,schema_filepath=SCHEMA_FILE_PATH):
        print(f"Config file path: {config_filepath}")
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.model_evaluation.root_dir])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        param = self.params.Models
        schema = self.schema.TARGET_COLUMN

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=Path(config.test_data_path),
            model_paths=config.model_paths,
            all_params=param,
            metrics_file_paths=config.metrics_file_paths,
            target_column=schema.name,
            mlflow_url="https://dagshub.com/Rutvik46/Heart-Failure-Prediction-with-Machine-Learning-MLflow.mlflow"
        )

        return model_evaluation_config

In [9]:
import pandas as pd
from mlProject import logger
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, precision_score, recall_score
from urllib.parse import urlparse
from sklearn.preprocessing import StandardScaler
import mlflow
import mlflow.sklearn
import numpy as np
import joblib
from mlProject.utils.common import save_json 
import time

In [ ]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def evaluation_metrics(self,actual,pred):
        Accuracy = accuracy_score(actual,pred)
        F1_Score = f1_score(actual,pred)
        Precision_Score = precision_score(actual,pred)
        Recall_Score = recall_score(actual,pred)
        AUC_Score = roc_auc_score(actual, pred)
        
        return Accuracy, F1_Score, Precision_Score, Recall_Score, AUC_Score

    def log_into_mlflow(self):

        # Load models
        SVM = joblib.load(Path(self.config.model_paths[0]))
        KNN = joblib.load(Path(self.config.model_paths[1]))
        AdaBoost = joblib.load(Path(self.config.model_paths[2]))

        models = {"Support_Vector_Machine":SVM,"Knearest_Neighbour":KNN, "AdaBoost":AdaBoost}

        # Load test data
        test_data = pd.read_csv(self.config.test_data_path)
        X_test = test_data.drop([self.config.target_column], axis=1)
        Y_test = test_data[self.config.target_column]

        for index, model_name in enumerate(models.keys()):
            mlflow.set_registry_uri(self.config.mlflow_url)
            tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

            with mlflow.start_run():

                predicted_qualities = models[model_name].predict(X_test)
        
                # Evaluation metrcis
                (Accuracy, F1_Score, Precision_Score, Recall_Score, AUC_Score) = self.evaluation_metrics(Y_test, predicted_qualities)
                
                # Saving metrics as local
                scores = {"Accuracy":Accuracy, "F1_Score":F1_Score, "Precision_Score":Precision_Score, "Recall_Score":Recall_Score, "AUC_Score":AUC_Score}
                save_json(path=Path(self.config.metrics_file_paths[index]), data=scores)

                '''
                mlflow.log_params(self.config.all_params[model_name])

                mlflow.log_metric("Accuracy",Accuracy)
                mlflow.log_metric("F1_Score",F1_Score)
                mlflow.log_metric("Precision_Score",Precision_Score)
                mlflow.log_metric("Recall_Score",Recall_Score)
                mlflow.log_metric("AUC_Score",AUC_Score)


                # Model registry does not work with file store
                if tracking_url_type_store != "file":

                    # Register the model
                    # There are other ways to use the Model Registry, which depends on the use case,
                    # please refer to the doc for more information:
                    # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                    mlflow.sklearn.log_model(models[model_name], "model", registered_model_name=model_name)
                else:
                    mlflow.sklearn.log_model(models[model_name], "model")'''
                
                time.sleep(7)



In [16]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.log_into_mlflow()

except Exception as e:
    logger.exception(e)
    raise e

Config file path: config\config.yaml
[2025-07-22 18:25:37,689 : INFO: yaml file: config\config.yaml loaded successfully]
[2025-07-22 18:25:37,691 : INFO: yaml file: params.yaml loaded successfully]
[2025-07-22 18:25:37,693 : INFO: yaml file: schema.yaml loaded successfully]
[2025-07-22 18:25:37,693 : INFO: created directory at: artifacts/model_evaluation]
[2025-07-22 18:25:37,909 : INFO: json file saved at: artifacts\model_evaluation\Support_Vector_Machine.json]


2025/07/22 18:25:43 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run awesome-seal-371 at: https://dagshub.com/Rutvik46/Heart-Failure-Prediction-with-Machine-Learning-MLflow.mlflow/#/experiments/0/runs/f13e0f9965e543d3ad9c75b770dd8cbd
🧪 View experiment at: https://dagshub.com/Rutvik46/Heart-Failure-Prediction-with-Machine-Learning-MLflow.mlflow/#/experiments/0
[2025-07-22 18:25:47,906 : ERROR: INTERNAL_ERROR: Response: {'error': 'unsupported endpoint, please contact support@dagshub.com'}]
Traceback (most recent call last):
  File "C:\Users\rutvi\AppData\Local\Temp\ipykernel_13208\2980524165.py", line 5, in <module>
    model_evaluation.log_into_mlflow()
  File "C:\Users\rutvi\AppData\Local\Temp\ipykernel_13208\1490570217.py", line 59, in log_into_mlflow
    mlflow.sklearn.log_model(models[model_name], "model", registered_model_name=model_name)
  File "c:\Users\rutvi\anaconda3\envs\mlProject\Lib\site-packages\mlflow\sklearn\__init__.py", line 426, in log_model
    return Model.log(
           ^^^^^^^^^^
  File "c:\Users\rutvi\anaconda3\envs\mlP

RestException: INTERNAL_ERROR: Response: {'error': 'unsupported endpoint, please contact support@dagshub.com'}